In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fm= pd.read_csv('../input/fmworkdata/dflast2.csv')



In [ ]:
drop_cols = [ 'GK','DRL',    'DC',   'WBRL', 'DM',   'MRL',  'MC',   'AMRL', 'AMC','SC']
fm = fm.drop(drop_cols, axis=1)

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car
cat_cols, num_cols, cat_but_car = grab_col_names(fm)


In [ ]:
NON_PLAYER_ATTRIBUTES = [' Pun ', ' Ref ', ' Com ', ' Aer ', ' Thr ', ' Han ', ' Ecc ', ' Kic ', ' Cmd ', ' 1v1 ', ' TRO ']
NON_GK_ATRIBUTES = [' L Th ', ' Lon ', ' Tck ', ' Cro ', ' Mar ', ' Cor ', ' Hea ', ' Dri ', ' Fin ']

MID = {'Advanced Playmaker (Support)': [['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'OtB', 'Tea', 'Vis'],
                                        ['Dri', 'Ant', 'Fla', 'Agi']],
       'Advanced Playmaker (Attack)': [['Dri', 'Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'OtB', 'Tea', 'Vis'],
                                       ['Ant', 'Fla', 'Agi', 'Acc']],
       'Anchor (Defend)': [['Mar', 'Tck', 'Ant', 'Cnt', 'Dec', 'Pos'],
                           ['Cmp', 'Tea', 'Str']],
       'Attacking Midfielder (Support)': [['Fir', 'Lon', 'Pas', 'Tec', 'Ant', 'Dec', 'Fla', 'OtB'],
                                          ['Dri', 'Cmp', 'Vis', 'Agi']],
       'Attacking Midfielder (Attack)': [['Dri', 'Fir', 'Lon', 'Pas', 'Tec', 'Ant', 'Dec', 'Fla', 'OtB'],
                                         ['Fin', 'Cmp', 'Vis', 'Agi']],
       'Ball Winning Midfielder (Defend)': [['Tck', 'Agg', 'Ant', 'Tea', 'Wor', 'Sta'],
                                            ['Mar', 'Bra', 'Cnt', 'Pos', 'Agi', 'Pac', 'Str']],
       'Ball Winning Midfielder (Support)': [['Tck', 'Agg', 'Ant', 'Tea', 'Wor', 'Sta'],
                                             ['Mar', 'Pas', 'Bra', 'Cnt', 'Agi', 'Pac', 'Str']],
       'Box to Box Midfielder (Support)': [['Pas', 'Tck', 'OtB', 'Tea', 'Wor', 'Sta'],
                                           ['Dri', 'Fin', 'Fir', 'Lon', 'Tec', 'Agg', 'Ant', 'Cmp', 'Dec', 'Pos', 'Acc', 'Bal', 'Pac', 'Str']],
       'Carrilero (Support)': [['Fir', 'Pas', 'Tck', 'Dec', 'Pos', 'Tea', 'Sta'],
                               ['Tec', 'Ant', 'Cmp', 'Cnt', 'OtB', 'Vis', 'Wor']],
       'Central Midfielder (Defend)': [['Tck', 'Cnt', 'Dec', 'Pos', 'Tea'],
                                       ['Fir', 'Mar', 'Pas', 'Tec', 'Agg', 'Ant', 'Cmp', 'Wor', 'Sta']],
       'Central Midfielder (Support)': [['Fir', 'Pas', 'Tck', 'Dec', 'Tea'],
                                        ['Tec', 'Ant', 'Cmp', 'Cnt', 'OtB', 'Vis', 'Wor', 'Sta']],
       'Central Midfielder (Attack)': [['Fir', 'Pas', 'Dec', 'OtB'],
                                       ['Lon', 'Tck', 'Tec', 'Ant', 'Cmp', 'Tea', 'Vis', 'Wor', 'Acc', 'Sta']],
       'Deep Lying Playmaker (Defend)': [['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Tea', 'Vis'],
                                         ['Tck', 'Ant', 'Pos', 'Bal']],
       'Deep Lying Playmaker (Support)': [['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Tea', 'Vis'],
                                          ['Ant', 'OtB', 'Pos', 'Bal']],
       'Defensive Midfielder (Defend)': [['Tck', 'Ant', 'Cnt', 'Pos', 'Tea'],
                                         ['Mar', 'Pas', 'Agg', 'Cmp', 'Dec', 'Wor', 'Sta', 'Str']],
       'Defensive Midfielder (Support)': [['Tck', 'Ant', 'Cnt', 'Pos', 'Tea'],
                                          ['Mar', 'Pas', 'Agg', 'Cmp', 'Dec', 'Wor', 'Sta', 'Str']],
       'Defensive Winger (Defend)' :[['OtB', 'Tec','Tea','Pos','Ant','Sta','Wor'],
                                          ['Agg','Tck','Cro','Mar','Cnt','Dec','Fir','Acc','Dri']],
       'Defensive Winger (Support)':[['OtB', 'Tec','Tea','Cro','Sta','Wor'],
                                          ['Agg','Tck','Cmp','Pos','Pas','Ant','Mar','Cnt','Dec','Fir','Acc','Dri']],
       'Enganche (Support)': [['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Vis'],
                              ['Dri', 'Ant', 'Fla', 'OtB', 'Tea', 'Agi']],
       'Half Back (Defend)': [['Mar', 'Tck', 'Ant', 'Cmp', 'Cnt', 'Dec', 'Pos', 'Tea'],
                              ['Fir', 'Pas', 'Agg', 'Bra', 'Wor', 'Jum', 'Sta', 'Str']],
       'Inside Forward (Support)': [['Dri', 'Fir', 'Pas', 'Tec', 'OtB', 'Acc', 'Agi', 'Bal'],
                                    ['Fin', 'Lon', 'Ant', 'Cmp', 'Fla', 'Vis', 'Pac']],
       'Inside Forward (Attack)': [['Dri', 'Fin', 'Fir', 'Tec', 'OtB', 'Acc', 'Agi', 'Bal'],
                                   ['Lon', 'Pas', 'Ant', 'Cmp', 'Fla', 'Pac']],
       'Inverted Winger (Support)': [['Dri', 'Pas', 'Tec', 'OtB', 'Acc'],
                                     ['Cro', 'Fir', 'Lon', 'Cmp', 'Dec', 'Vis', 'Wor', 'Agi', 'Pac', 'Sta']],
       'Inverted Winger (Attack)': [['Dri', 'Pas', 'Tec', 'OtB', 'Acc', 'Agi'],
                                    ['Cro', 'Fir', 'Lon', 'Ant', 'Cmp', 'Dec', 'Fla', 'Vis', 'Pac']],
       'Mezzala (Support)': [['Pas', 'Tec', 'Dec', 'OtB', 'Wor', 'Acc'],
                             ['Dri', 'Fir', 'Lon', 'Tck', 'Ant', 'Cmp', 'Vis', 'Bal', 'Sta']],
       'Mezzala (Attack)': [['Dri', 'Pas', 'Tec', 'Dec', 'OtB', 'Vis', 'Wor', 'Acc'],
                            ['Fin', 'Fir', 'Lon', 'Ant', 'Cmp', 'Fla', 'Bal', 'Sta']],
       'Raumdeuter (Support)': [['Fin', 'Ant', 'Cmp', 'Cnt', 'Dec', 'OtB', 'Bal'],
                                ['Fir', 'Tec', 'Wor', 'Acc', 'Sta']],
       'Regista (Support)': [['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Fla', 'OtB', 'Tea', 'Vis'],
                             ['Dri', 'Lon', 'Ant', 'Bal']],
       'Roaming Playmaker (Support)': [['Fir', 'Pas', 'Tec', 'Ant', 'Cmp', 'Dec', 'OtB', 'Tea', 'Vis', 'Wor', 'Acc', 'Sta'],
                                       ['Dri', 'Lon', 'Cnt', 'Pos', 'Agi', 'Bal', 'Pac']],
       'Segundo Volante (Support)': [['Mar', 'Pas', 'Tck', 'OtB', 'Pos', 'Wor', 'Pac', 'Sta'],
                                     ['Fin', 'Fir', 'Lon', 'Ant', 'Cmp', 'Cnt', 'Dec', 'Acc', 'Bal', 'Str']],
       'Segundo Volante (Attack)': [['Fin', 'Lon', 'Pas', 'Tck', 'Ant', 'OtB', 'Pos', 'Wor', 'Pac', 'Sta'],
                                    ['Fir', 'Mar', 'Cmp', 'Cnt', 'Dec', 'Acc', 'Bal', 'Str']],
       'Shadow Striker (Attack)': [['Dri', 'Fin', 'Fir', 'Ant', 'Cmp', 'OtB', 'Acc'],
                                    ['Pas', 'Tec', 'Cnt', 'Dec', 'Wor', 'Agi', 'Bal', 'Pac', 'Sta']],
       'Wide Midfielder (Defend)': [['Pas', 'Tck', 'Cnt', 'Dec', 'Pos', 'Tea', 'Wor'],
                                    ['Cro', 'Fir', 'Mar', 'Tec', 'Ant', 'Cmp', 'Sta']],
       'Wide Midfielder (Support)': [['Pas', 'Tck', 'Dec', 'Tea', 'Wor', 'Sta'],
                                     ['Cro', 'Fir', 'Tec', 'Ant', 'Cmp', 'Cnt', 'OtB', 'Pos', 'Vis']],
       'Wide Midfielder (Attack)': [['Cro', 'Fir', 'Pas', 'Dec', 'Tea', 'Wor', 'Sta'],
                                    ['Tck', 'Tec', 'Ant', 'Cmp', 'OtB', 'Vis']],
       'Wide Playmaker (Support)': [['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Tea', 'Vis'],
                                    ['Dri', 'OtB', 'Agi']],
       'Wide Playmaker (Attack)': [['Dri', 'Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'OtB', 'Tea', 'Vis'],
                                   ['Ant', 'Fla', 'Acc', 'Agi']],
       'Wide Target Forward (Support)': [['Hea', 'Bra', 'Tea', 'Jum', 'Str'],
                                         ['Cro', 'Fir', 'Ant', 'OtB', 'Wor', 'Bal', 'Sta']],
       'Wide Target Forward (Attack)': [['Hea', 'Bra', 'OtB', 'Jum', 'Str'],
                                        ['Cro', 'Fin', 'Fir', 'Ant', 'Tea', 'Wor', 'Bal', 'Sta']],
       'Winger (Support)': [['Cro', 'Dri', 'Tec', 'OtB', 'Acc', 'Pac'],
                            ['Fir', 'Pas', 'Wor', 'Agi', 'Sta']],
       'Winger (Attack)': [['Cro', 'Dri', 'Tec', 'OtB', 'Acc', 'Pac'],
                           ['Fir', 'Pas', 'Ant', 'Fla', 'Agi']]
       }










In [ ]:
GK={
    
    'Sweeper Keeper (Defend)':  [[' Ref ', ' Pos ', ' Ant ', ' Cnt ', ' Kic ', ' Agi ', ' Cmd ', ' 1v1 '],
    [' Vis ', ' Cmp ', ' Pas ', ' Dec ', ' Fir ', ' Com ', ' Acc ', ' Aer ', ' Thr ', ' Han ', ' TRO ']],
   
    'Sweeper Keeper (Support)': [[' Cmp ', ' Ref ', ' Pos ', ' Ant ', ' Cnt ', ' Kic ', ' Agi ', ' Cmd ', ' 1v1 ', ' TRO '],
    [' Vis ', ' Pas ', ' Dec ', ' Fir ', ' Com ', ' Acc ', ' Aer ', ' Thr ', ' Han ']],

    'Sweeper Keeper (Attack)': [[' Cmp ', ' Ref ', ' Pos ', ' Ant ', ' Cnt ', ' Kic ', ' Agi ', ' Cmd ', ' 1v1 ', ' TRO '],
    [' Vis ', ' Pas ', ' Dec ', ' Fir ', ' Com ', ' Acc ', ' Aer ', ' Thr ', ' Han ', ' Ecc ']],

    'Goalkeeper (Defend)': [[' Ref ', ' Pos ', ' Cnt ', ' Com ', ' Aer ', ' Han ', ' Kic ', ' Agi ', ' Cmd '],
    [' Ant ', ' Dec ', ' Thr ', ' 1v1 ']]
    }

FW={'Complete Forward (Attack)': [[' OtB ', ' Tec ', ' Cmp ', ' Ant ', ' Hea ', ' Fir ', ' Acc ', ' Str ', ' Dri ', ' Agi ', ' Fin '],
    [' Jum ', ' Vis ', ' Lon ', ' Tea ', ' Pas ', ' Dec ', ' Pac ', ' Bal ', ' Sta ', ' Wor ']],

    'Complete Forward (Support)': [[' Vis ', ' Lon ', ' OtB ', ' Tec ', ' Cmp ', ' Pas ', ' Ant ', ' Dec ', ' Hea ', ' Fir ', ' Acc ', ' Str ', ' Dri ', ' Agi '],
    [' Jum ', ' Tea ', ' Pac ', ' Bal ', ' Sta ', ' Wor ', ' Fin ']],

    'Target Forward (Support)': [[' Jum ', ' Tea ', ' Hea ', ' Str ', ' Bal ', ' Bra '],
    [' Agg ', ' OtB ', ' Cmp ', ' Ant ', ' Dec ', ' Fir ', ' Fin ']],

    'Target Forward (Attack)': [[' Jum ', ' OtB ', ' Cmp ', ' Hea ', ' Str ', ' Bal ', ' Bra ', ' Fin '],
    [' Agg ', ' Tea ', ' Ant ', ' Dec ', ' Fir ']],

    'Pressing Forward (Defend)': [[' Agg ', ' Tea ', ' Ant ', ' Dec ', ' Acc ', ' Pac ', ' Sta ', ' Wor ', ' Bra '],
    [' Cmp ', ' Cnt ', ' Fir ', ' Str ', ' Bal ', ' Agi ']],

    'Pressing Forward (Support)': [[' Agg ', ' Tea ', ' Ant ', ' Dec ', ' Acc ', ' Pac ', ' Sta ', ' Wor ', ' Bra '],
    [' OtB ', ' Cmp ', ' Pas ', ' Cnt ', ' Fir ', ' Str ', ' Bal ', ' Agi ']],

    'Pressing Forward (Attack)': [[' Agg ', ' OtB ', ' Tea ', ' Ant ', ' Acc ', ' Pac ', ' Sta ', ' Wor ', ' Bra '],
    [' Cmp ', ' Cnt ', ' Dec ', ' Fir ', ' Str ', ' Bal ', ' Agi ', ' Fin ']],

    'Deep Lying Forward (Support)': [[' OtB ', ' Tec ', ' Tea ', ' Cmp ', ' Pas ', ' Dec ', ' Fir '],
    [' Vis ', ' Fla ', ' Ant ', ' Str ', ' Bal ', ' Fin ']],

    'Deep Lying Forward (Attack)': [[' OtB ', ' Tec ', ' Tea ', ' Cmp ', ' Pas ', ' Dec ', ' Fir '],
    [' Vis ', ' Fla ', ' Ant ', ' Str ', ' Dri ', ' Bal ', ' Fin ']],

    'Poacher (Attack)': [[' OtB ', ' Cmp ', ' Ant ', ' Fin '],
    [' Tec ', ' Dec ', ' Hea ', ' Fir ', ' Acc ']],

    'Advanced Forward (Attack)': [[' OtB ', ' Tec ', ' Cmp ', ' Fir ', ' Acc ', ' Dri ', ' Fin '],
    [' Pas ', ' Ant ', ' Dec ', ' Pac ', ' Bal ', ' Sta ', ' Agi ', ' Wor ']],

    'Trequartista (Attack)': [[' Vis ', ' OtB ', ' Tec ', ' Cmp ', ' Pas ', ' Fla ', ' Dec ', ' Fir ', ' Acc ', ' Dri '],
    [' Ant ', ' Bal ', ' Agi ', ' Fin ']],

    'False Nine (Support)': [[' Vis ', ' OtB ', ' Tec ', ' Cmp ', ' Pas ', ' Dec ', ' Fir ', ' Acc ', ' Dri ', ' Agi '],
    [' Tea ', ' Fla ', ' Ant ', ' Bal ', ' Fin ']]
}

In [ ]:
DEF = {"Ball Playing Defender (Defender)": [[" Jum ", " Tck ", " Cmp ", " Pos ", " Pas ", " Mar ", " Hea ", " Str "],
                                           [" Agg ", " Vis ", " Tec ", " Ant ", " Cnt ", " Dec ", " Fir ", " Pac ", " Bra "]],
       "Ball Playing Defender (Stopper)": [[" Agg ", " Jum ", " Tck ", " Cmp ", " Pos ", " Pas ", " Dec ", " Hea ", " Str ", " Bra "],
                                          [" Vis ", " Tec ", " Ant ", " Mar ", " Cnt ", " Fir "]],
       "Ball Playing Defender (Cover)": [[" Tck ", " Cmp ", " Pos ", " Pas ", " Ant ", " Mar ", " Cnt ", " Dec ", " Pac "],
                                        [" Jum ", " Vis ", " Tec ", " Hea ", " Fir ", " Str ", " Bra "]],

       "No-Nonsense Centre-Back (Defend)": [[" Jum ", " Tck ", " Pos ", " Mar ", " Hea ", " Str "],
                                           [" Agg ", " Ant ", " Cnt ", " Pac ", " Bra "]],
       "No-Nonsense Centre-Back (Stopper)": [[" Agg ", " Jum ", " Tck ", " Pos ", " Hea ", " Str ", " Bra "],
                                            [" Ant ", " Mar ", " Cnt "]],
       "No-Nonsense Centre-Back (Cover)": [[" Tck ", " Pos ", " Ant ", " Mar ", " Cnt ", " Pac "],
                                          [" Jum ", " Hea ", " Str ", " Bra "]],

       "Wide Centre-Back (Defend)": [[" Jum ", " Tck ", " Pos ", " Cro ", " Mar ", " Hea ", " Str ", " Sta "],
                                    [" Agg ", " Cmp ", " Ant ", " Cnt ", " Dec ", " Pac ", " Dri ", " Wor ", " Bra "]],
       "Wide Centre-Back (Support)": [[' Jum ', ' Tck ', ' Pos ', ' Cro ', ' Mar ', ' Hea ', ' Pac ', ' Str ', ' Dri ', ' Sta '],
                                     [' Agg ', ' OtB ', ' Cmp ', ' Ant ', ' Cnt ', ' Dec ', ' Wor ', ' Bra ']],
       "Wide Centre-Back (Attack)": [[' Jum ', ' OtB ', ' Tck ', ' Cro ', ' Mar ', ' Hea ', ' Pac ', ' Str ', ' Dri ', ' Sta '],
                                    [' Agg ', ' Cmp ', ' Pos ', ' Ant ', ' Cnt ', ' Dec ', ' Wor ', ' Bra ']],

       "Libero (Support)": [[' Vis ', ' Tck ', ' Tea ', ' Cmp ', ' Pos ', ' Pas ', ' Ant ', ' Mar ', ' Cnt ', ' Dec ', ' Fir ', ' Pac '],
                           [' Jum ', ' Tec ', ' Fla ', ' Hea ', ' Str ', ' Dri ', ' Bal ', ' Sta ', ' Agi ', ' Bra ']],
       "Libero (Attack)": [[' Vis ', ' Tck ', ' Tea ', ' Cmp ', ' Pos ', ' Pas ', ' Fla ', ' Ant ', ' Mar ', ' Cnt ', ' Dec ', ' Fir ', ' Pac ', ' Dri '],
                          [' Jum ', ' Lon ', ' Tec ', ' Hea ', ' Acc ', ' Str ', ' Bal ', ' Sta ', ' Agi ', ' Bra ']],

       "Central Defender (Defend)": [[' Jum ', ' Tck ', ' Pos ', ' Mar ', ' Hea ', ' Str '],
                                    [' Agg ', ' Cmp ', ' Ant ', ' Cnt ', ' Dec ', ' Pac ', ' Bra ']],
       "Central Defender (Stopper)": [[' Agg ', ' Jum ', ' Tck ', ' Pos ', ' Dec ', ' Hea ', ' Str ', ' Bra '],
                                     [' Cmp ', ' Ant ', ' Mar ', ' Cnt ']],
       "Central Defender (Cover)": [[' Tck ', ' Pos ', ' Ant ', ' Mar ', ' Cnt ', ' Dec ', ' Pac '],
                                   [' Jum ', ' Cmp ', ' Hea ', ' Str ', ' Bra ']],

       "Full-Back (Defend)": [[' Tck ', ' Pos ', ' Ant ', ' Mar ', ' Cnt '],
                             [' Tea ', ' Cmp ', ' Pas ', ' Cro ', ' Dec ', ' Pac ', ' Sta ']],
       "Full-Back (Support)": [[' Tck ', ' Tea ', ' Pos ', ' Ant ', ' Mar ', ' Cnt ', ' Wor '],
                              [' Tec ', ' Cmp ', ' Pas ', ' Cro ', ' Dec ', ' Pac ', ' Dri ', ' Sta ']],
       "Full-Back (Attack)": [[' Tck ', ' Tea ', ' Pos ', ' Ant ', ' Cro ', ' Pac ', ' Sta ', ' Wor '],
                             [' OtB ', ' Tec ', ' Cmp ', ' Pas ', ' Mar ', ' Cnt ', ' Dec ', ' Fir ', ' Acc ', ' Dri ',
                              ' Agi ']],
       "Full-Back (Automatic)": [[' Tck ', ' Tea ', ' Pos ', ' Ant ', ' Mar ', ' Cnt ', ' Wor '],
                                [' Tec ', ' Cmp ', ' Pas ', ' Cro ', ' Dec ', ' Pac ', ' Dri ', ' Sta ']],

       "Wing-Back (Defend)": [[' Tck ', ' Tea ', ' Pos ', ' Ant ', ' Mar ', ' Acc ', ' Sta ', ' Wor '],
                             [' OtB ', ' Tec ', ' Pas ', ' Cro ', ' Cnt ', ' Dec ', ' Fir ', ' Pac ', ' Dri ',
                              ' Agi ']],
       "Wing-Back (Support)": [[' OtB ', ' Tck ', ' Tea ', ' Cro ', ' Mar ', ' Acc ', ' Dri ', ' Sta ', ' Wor '],
                              [' Tec ', ' Pos ', ' Pas ', ' Ant ', ' Cnt ', ' Dec ', ' Fir ', ' Pac ', ' Agi ']],
       "Wing-Back (Attack)": [[' OtB ', ' Tck ', ' Tec ', ' Tea ', ' Cro ', ' Acc ', ' Pac ', ' Dri ', ' Sta ', ' Wor '],
                             [' Pos ', ' Pas ', ' Fla ', ' Ant ', ' Mar ', ' Cnt ', ' Dec ', ' Fir ', ' Agi ']],
       "Wing-Back (Automatic)": [[' OtB ', ' Tck ', ' Tea ', ' Cro ', ' Mar ', ' Acc ', ' Dri ', ' Sta ', ' Wor '],
                                [' Tec ', ' Pos ', ' Pas ', ' Ant ', ' Cnt ', ' Dec ', ' Fir ', ' Pac ', ' Agi ']],

       "No-Nonsense Full-Back (Defend)": [[' Tck ', ' Pos ', ' Ant ', ' Mar ', ' Cnt ', ' Str '],
                                         [' Agg ', ' Tea ', ' Hea ', ' Bra ']],

       "Complete Wing-Back (Support)": [[' OtB ', ' Tec ', ' Tea ', ' Pas ', ' Cro ', ' Dec ', ' Fir ', ' Acc ', ' Pac ', ' Dri ', ' Sta ', ' Wor '],
                                       [' Tck ', ' Cmp ', ' Fla ', ' Ant ', ' Bal ', ' Agi ']],
       "Complete Wing-Back (Attack)": [[' OtB ', ' Tec ', ' Tea ', ' Pas ', ' Fla ', ' Cro ', ' Dec ', ' Fir ', ' Acc ', ' Pac ', ' Dri ', ' Sta ',' Wor '],
                                      [' Tck ', ' Cmp ', ' Ant ', ' Bal ', ' Agi ']],

       "Inverted Wing-Back (Defend)": [[' Tck ', ' Tea ', ' Pos ', ' Pas ', ' Ant ', ' Mar ', ' Dec ', ' Wor '],
                                      [' OtB ', ' Tec ', ' Cnt ', ' Fir ', ' Acc ', ' Dri ', ' Sta ', ' Agi ']],
       "Inverted Wing-Back (Support)": [[' OtB ', ' Tck ', ' Tea ', ' Pas ', ' Mar ', ' Dec ', ' Sta ', ' Wor '],
                                       [' Tec ', ' Cmp ', ' Pos ', ' Ant ', ' Cnt ', ' Fir ', ' Acc ', ' Dri ',
                                        ' Agi ']],
       "Inverted Wing-Back (Attack)": [[' OtB ', ' Tck ', ' Tec ', ' Tea ', ' Pas ', ' Mar ', ' Dec ', ' Acc ', ' Dri ', ' Sta ', ' Wor '],
                                      [' Lon ', ' Cmp ', ' Pos ', ' Fla ', ' Ant ', ' Cnt ', ' Fir ', ' Pac ', ' Agi ']],
       "Inverted Wing-Back (Automatic)": [[' OtB ', ' Tck ', ' Tea ', ' Pas ', ' Mar ', ' Dec ', ' Sta ', ' Wor '],
                                         [' Tec ', ' Cmp ', ' Pos ', ' Ant ', ' Cnt ', ' Fir ', ' Acc ', ' Dri ', ' Agi ']]
       }

In [ ]:
scores = pd.DataFrame()
scores['UID'] = fm['UID']
scores['Name'] = fm["Name"]
scores.head()
all_attributes = []

positions = {'GK': GK, 'DEF': DEF, 'MID': MID, 'FW': FW}

for key, value in GK.items():
    GK[key] = {'key': value[0], 'preferred':value[1]}
for key, value in DEF.items():
    DEF[key] = {'key': value[0], 'preferred':value[1]}
for key, value in MID.items():
    MID[key] = {'key': value[0], 'preferred':value[1]}
for key, value in FW.items():
    FW[key] = {'key': value[0], 'preferred':value[1]}


In [ ]:
for key, value in positions.items():
    for role, attributes in value.items():
        all_attributes.extend(attributes['key'])
        all_attributes.extend(attributes['preferred'])
all_attributes = list(set(all_attributes))
len(all_attributes)
       
all_attributes = [x.strip(' ') for x in all_attributes]


In [ ]:
for pos_name, roles in positions.items():
    for role_name, attributes in roles.items():
        keys_22 = attributes['key']
        preferred_22 = attributes['preferred']
        keys_22 = [x.strip(' ') for x in keys_22]
        preferred_22 = [x.strip(' ') for x in preferred_22]
        keys_21 = [x + '_21' for x in keys_22]
        keys_20 = [x + '_20' for x in keys_22]
        preferred_21 = [x + '_21' for x in preferred_22]
        preferred_20 = [x + '_20' for x in preferred_22]

        if pos_name == 'GK':
            others_22 = [x for x in all_attributes if x not in keys_22 and x not in preferred_22 and x not in NON_GK_ATRIBUTES]
        else:
            others_22 = [x for x in all_attributes if x not in keys_22 and x not in preferred_22 and x not in NON_PLAYER_ATTRIBUTES]

        others_21 = [x + '_21' for x in others_22]
        others_20 = [x + '_20' for x in others_22]
        scores[role_name] = (fm[keys_22].mean(axis=1) * 0.55 + fm[preferred_22].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.6\
                            +(fm[keys_21].mean(axis=1) * 0.55 + fm[preferred_21].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.3\
                            +(fm[keys_20].mean(axis=1) * 0.55 + fm[preferred_20].mean(axis=1) * 0.35+ fm[others_22].mean(axis=1)*0.1)*0.1

        print(f"{pos_name} {role_name} done")



In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 100))
scores.iloc[:,2:] = pd.DataFrame(scaler.fit_transform(scores.iloc[:,2:]), columns=scores.iloc[:,2:].columns)


In [ ]:
scores.sort_values(by='Trequartista (Attack)', ascending=False, na_position='first').head()


In [ ]:
scores.to_csv("scores.csv",index=False)